In [1]:
import sqlite3
import pandas as pd
import numpy as np

In [2]:
conn = sqlite3.connect('yelp_dataset_business.db') 
new = pd.read_sql("""SELECT * FROM business""",conn )
new.shape

(150346, 14)

In [3]:
conn.close()